**Cost calculator using Tensorflow**

In [ ]:
import tensorflow as tf

class compute_cost_by_gpu:
    def __init__(self, path_lenght):
        self.inp_p = tf.placeholder(tf.float32, shape=[None,path_lenght], name='input_p')
        self.inp_x = tf.placeholder(tf.float32, shape=[None,path_lenght], name='input_x' )
        self.inp_y = tf.placeholder(tf.float32, shape=[None,path_lenght], name='input_y')
        penalty10_srouce = np.zeros(path_lenght-1)
        for i in range(0, path_lenght-1):
            if (i%10)==9 : 
                penalty10_srouce[i] = 0.1
        penalty10 = tf.constant(penalty10_srouce,dtype=tf.float32)
        dx = tf.subtract( self.inp_x[:,:path_lenght-1], self.inp_x[:,1:] )
        dy = tf.subtract( self.inp_y[:,:path_lenght-1], self.inp_y[:,1:] )
        distance = tf.sqrt( tf.add( tf.multiply(dx,dx), tf.multiply(dy,dy) ) )
        weight   = tf.add( tf.multiply( self.inp_p[:,:path_lenght-1], penalty10 ), tf.constant(1.0,dtype=tf.float32) )
        distance_weighted = tf.multiply( distance, weight  )
        self.total_score = tf.reduce_sum( distance_weighted, axis=1 )
        #print( dx.shape )
        #print( distance.shape )
        #print( self.total_score.shape )
        self.sess = tf.Session()
        
    def compute_cost(self, is_not_prime, x, y):        
        return self.sess.run([self.total_score],feed_dict={self.inp_p:is_not_prime,self.inp_x:x,self.inp_y:y})

**Function verification**

CPU version
* using initial path - generated by Concode TSP Solver

In [ ]:
import numpy as np
import pandas as pd
import time
from sympy import isprime, primerange

# "Initial score (generated by Concorde)
initial_path = pd.read_csv('../input/santa2018/santa2018_initial.csv')

cities = pd.read_csv("../input/traveling-santa-2018-prime-paths/cities.csv")
pnums = [i for i in primerange(0, 197770)]
path_df = cities.reindex(initial_path['Path']).reset_index()    

def score_path():
    path_df['step'] = np.sqrt((path_df.X - path_df.X.shift())**2 + 
                              (path_df.Y - path_df.Y.shift())**2)
    path_df['step_adj'] = np.where((path_df.index) % 10 != 0,
                                   path_df.step,
                                   path_df.step + 
                                   path_df.step*0.1*(~path_df.CityId.shift().isin(pnums)))
    return path_df.step_adj.sum()

print( "Cost - computed by CPU : " + str( score_path() ) )

GPU version
* It is just function verification, not for performance evaluation. So... 1 path / 1 batch

In [ ]:
tf_cities = pd.read_csv("../input/traveling-santa-2018-prime-paths/cities.csv")
tf_cities['isPrime'] = tf_cities.CityId.apply(isprime)
tf_cities['isNotPrime'] = np.invert(tf_cities['isPrime']).astype(float)
tf_path_df = tf_cities.reindex(initial_path['Path']).reset_index()       
tf_input_p = [tf_path_df.isNotPrime]
tf_input_x = [tf_path_df.X         ]
tf_input_y = [tf_path_df.Y         ]
gpu_cost = compute_cost_by_gpu(len(initial_path['Path']))
print( "Cost - computed by GPU " + str( gpu_cost.compute_cost( tf_input_p, tf_input_x, tf_input_y )[0]) )

Check the result with the ground truth by summitting

In [ ]:
!cp ../input/santa2018/santa2018_initial.csv .

**Performance**

In [ ]:
t = time.time()
for i in range(1000):
    score_path()
print(time.time() - t)

In [ ]:
t = time.time()
tf_input_p = np.empty( (1000,len(tf_path_df)))
tf_input_x = np.empty( (1000,len(tf_path_df)))
tf_input_y = np.empty( (1000,len(tf_path_df)))
for i in range(1000):
    tf_input_p[i] = tf_path_df.isNotPrime
    tf_input_x[i] = tf_path_df.X         
    tf_input_y[i] = tf_path_df.Y         
gpu_cost.compute_cost( tf_input_p, tf_input_x, tf_input_y )
print(time.time() - t)